In [ ]:
# 1. 이전 설치 제거
!rm -rf /opt/chrome
!rm -f /usr/bin/chromedriver
!rm -f chrome-linux64.zip chromedriver-linux64.zip
!rm -rf chrome-linux64 chromedriver-linux64

# 2. Chrome 138 설치
!wget -q https://storage.googleapis.com/chrome-for-testing-public/138.0.7204.92/linux64/chrome-linux64.zip
!unzip -q chrome-linux64.zip
!mv chrome-linux64 /opt/chrome

# 3. ChromeDriver 138 설치
!wget -q https://storage.googleapis.com/chrome-for-testing-public/138.0.7204.92/linux64/chromedriver-linux64.zip
!unzip -q chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

# 4. Selenium 설치
!pip install -q selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 30.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [ ]:
# 저장 디렉토리 구성
BASE_DIR = "saved_html"
CATEGORY_DIR = os.path.join(BASE_DIR, "category_pages")
os.makedirs(CATEGORY_DIR, exist_ok=True)

# 셀레니움 드라이버 설정
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

In [ ]:
def get_categories(main_url):
    driver.get(main_url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    category_tags = soup.find_all("li", class_="displayCategoryImageMenu_list__eEKrH")

    categories = []
    for li in category_tags:
        name_tag = li.find("span", class_="displayCategoryImageMenu_title__Wizki")
        name = name_tag.get_text(strip=True) if name_tag else None

        button_tag = li.find("button")
        relative_url = button_tag.get("data-shp-contents-id") if button_tag else None

        # 정렬 조건 강제 추가
        full_url = None
        if relative_url:
            base_url = "https://shopping.naver.com" + relative_url
            if "sort=" in base_url:
                full_url = re.sub(r"sort=[^&]+", "sort=DISPLAY_CATEGORY_GENDER_AGE_GROUP_F20", base_url)
            else:
                full_url = base_url + "?sort=DISPLAY_CATEGORY_GENDER_AGE_GROUP_F20"

        if name and full_url:
            categories.append({
                "name": name.replace("/", "_"),
                "url": full_url
            })

    return categories

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

def scroll_and_save_html_by_items(name, url, max_items=500, delay=2, timeout=20):
    print(f"카테고리: {name} 시작 → {url}")
    driver.get(url)

    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CLASS_NAME, "displayCategoryProductCard_display_category_product_card__od27_"))
        )
    except:
        print(f"상품 초기 로딩 실패: {name}")
        return

    actions = ActionChains(driver)
    scrolls = 0
    prev_count = 0

    while True:
        cards = driver.find_elements(By.CLASS_NAME, "displayCategoryProductCard_display_category_product_card__od27_")
        count = len(cards)

        print(f"스크롤 {scrolls+1}회 | 상품 수: {count}")

        if count >= max_items:
            break
        if count == prev_count:
            print("더 이상 상품이 추가되지 않음")
            break

        try:
            # 마지막 상품 요소에 마우스를 이동 → 인터섹션 옵저버 발동 유도
            last_card = cards[-1]
            actions.move_to_element(last_card).perform()
        except Exception as e:
            print(f"스크롤 실패: {e}")
            break

        time.sleep(delay)
        prev_count = count
        scrolls += 1

    # HTML 저장
    os.makedirs("saved_html/category_pages", exist_ok=True)
    filename = os.path.join("saved_html/category_pages", f"{name}.html")
    with open(filename, "w", encoding="utf-8") as f:
        f.write(driver.page_source)
    print(f"저장 완료: {filename}")

In [ ]:
MAIN_URL = "https://shopping.naver.com/window/fashion-group/category/20006491?sort=DISPLAY_CATEGORY_GENDER_AGE_GROUP_F20"
categories = get_categories(MAIN_URL)

for cat in categories[:5]:  # 테스트로 앞 5개만 실행
    try:
        scroll_and_save_html_by_items(name=cat["name"], url=cat["url"])
    except Exception as e:
        print(f"{cat['name']} 처리 중 오류 발생: {e}")

카테고리: 전체 시작 → https://shopping.naver.com/window/fashion-group/category/20006491?sort=DISPLAY_CATEGORY_GENDER_AGE_GROUP_F20
스크롤 1회 | 상품 수: 40
스크롤 2회 | 상품 수: 60
스크롤 3회 | 상품 수: 55
스크롤 4회 | 상품 수: 55
더 이상 상품이 추가되지 않음
저장 완료: saved_html/category_pages/전체.html
카테고리: 스킨케어 시작 → https://shopping.naver.com/window/fashion-group/category/20006492?sort=DISPLAY_CATEGORY_GENDER_AGE_GROUP_F20
스크롤 1회 | 상품 수: 15
스크롤 2회 | 상품 수: 40
스크롤 3회 | 상품 수: 60
스크롤 4회 | 상품 수: 55
스크롤 5회 | 상품 수: 55
더 이상 상품이 추가되지 않음
저장 완료: saved_html/category_pages/스킨케어.html
카테고리: 선케어 시작 → https://shopping.naver.com/window/fashion-group/category/20006505?sort=DISPLAY_CATEGORY_GENDER_AGE_GROUP_F20
스크롤 1회 | 상품 수: 15
스크롤 2회 | 상품 수: 35
스크롤 3회 | 상품 수: 55
스크롤 4회 | 상품 수: 55
더 이상 상품이 추가되지 않음
저장 완료: saved_html/category_pages/선케어.html
카테고리: 마스크_팩 시작 → https://shopping.naver.com/window/fashion-group/category/20006513?sort=DISPLAY_CATEGORY_GENDER_AGE_GROUP_F20
스크롤 1회 | 상품 수: 15
스크롤 2회 | 상품 수: 40
스크롤 3회 | 상품 수: 60
스크롤 4회 | 상품 수: 55
스크롤 5회 | 상품 수: 55
더

테스트용

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# 셀레니움 드라이버 설정
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

# ✅ 1. 카테고리 URL 수집 함수
def get_categories(main_url):
    driver.get(main_url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    category_tags = soup.find_all("li", class_="displayCategoryImageMenu_list__eEKrH")

    categories = []
    for li in category_tags:
        name_tag = li.find("span", class_="displayCategoryImageMenu_title__Wizki")
        name = name_tag.get_text(strip=True) if name_tag else None

        button_tag = li.find("button")
        relative_url = button_tag.get("data-shp-contents-id") if button_tag else None
        full_url = "https://shopping.naver.com" + relative_url if relative_url else None

        if name and full_url:
            categories.append({"name": name.replace("/", "_"), "url": full_url})
    return categories

# ✅ 2. 카테고리 URL 수집 및 출력
MAIN_URL = "https://shopping.naver.com/window/fashion-group/category/20006491?sort=DISPLAY_CATEGORY_GENDER_AGE_GROUP_F20"
category_list = get_categories(MAIN_URL)

print("📌 수집된 카테고리 URL:")
for cat in category_list:
    print(f"{cat['name']} → {cat['url']}")

# ✅ 3. 하나의 카테고리 URL 테스트
test_url = category_list[1]["url"]  # 예: 2번째 카테고리 사용
driver.get(test_url)
time.sleep(5)

# ✅ 4. 해당 li 태그 존재 여부 확인
soup = BeautifulSoup(driver.page_source, "html.parser")
card_wrapper = soup.find("li", class_="displayCategoryCardListWrapper_display_category_card_list_wrapper__HthSR")

if card_wrapper:
    print("✅ 상품 카드 wrapper 태그를 찾았습니다:")
    print(card_wrapper.prettify())
else:
    print("❌ 상품 카드 wrapper 태그를 찾지 못했습니다.")

driver.quit()


📌 수집된 카테고리 URL:
전체 → https://shopping.naver.com/window/fashion-group/category/20006491
스킨케어 → https://shopping.naver.com/window/fashion-group/category/20006492
선케어 → https://shopping.naver.com/window/fashion-group/category/20006505
마스크_팩 → https://shopping.naver.com/window/fashion-group/category/20006513
클렌징 → https://shopping.naver.com/window/fashion-group/category/20006522
메이크업 → https://shopping.naver.com/window/fashion-group/category/20006536
네일케어 → https://shopping.naver.com/window/fashion-group/category/20006567
바디케어 → https://shopping.naver.com/window/fashion-group/category/20006585
헤어케어 → https://shopping.naver.com/window/fashion-group/category/20006612
뷰티소품 → https://shopping.naver.com/window/fashion-group/category/20006651
향수 → https://shopping.naver.com/window/fashion-group/category/20006681
남성화장품 → https://shopping.naver.com/window/fashion-group/category/20006688
뷰티디바이스 → https://shopping.naver.com/window/fashion-group/category/20006713
유아동화장품 → https://shopping.naver.com/w

review

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

# 셀레니움 설정
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# 상품 상세페이지 접속
url = "https://shopping.naver.com/window-products/beauty/7713179801"
driver.get(url)
time.sleep(3)

# 리뷰 탭 클릭
try:
    review_tab = driver.find_element(By.CSS_SELECTOR, 'a[href*="#REVIEW"]')
    review_tab.click()
    time.sleep(3)
except:
    print("리뷰 탭 클릭 실패")

# 스크롤로 리뷰 불러오기
for _ in range(2):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 리뷰 항목 가져오기
review_items = driver.find_elements(By.CSS_SELECTOR, 'li.BnwL_cs1av._nlog_click._nlog_impression_element')

# 첫 번째 리뷰의 HTML 출력
if review_items:
    print("리뷰 HTML 예시:")
    print(review_items[0].get_attribute('outerHTML'))
else:
    print("리뷰 항목이 발견되지 않았습니다.")

driver.quit()


리뷰 HTML 예시:
<li class="BnwL_cs1av _nlog_click _nlog_impression_element" data-shp-page-key="100399929" data-shp-sti="" data-shp-nsc="shoppingw.beauty" data-shp-inventory="revlist" data-shp-filter_con="[{&quot;key&quot;:&quot;리뷰 유형&quot;,&quot;value&quot;:&quot;전체보기&quot;},{&quot;key&quot;:&quot;리뷰 토픽&quot;,&quot;value&quot;:&quot;&quot;},{&quot;key&quot;:&quot;pgn&quot;,&quot;value&quot;:&quot;1&quot;}]" data-shp-order_con="[{&quot;key&quot;:&quot;리뷰 정렬&quot;,&quot;value&quot;:&quot;랭킹순&quot;}]" data-shp-area="revlist.review" data-shp-area-type="slot" data-shp-area-id="review" data-shp-contents-id="4748785097" data-shp-contents-type="review" data-shp-contents-grp="event" data-shp-contents-rank="0" data-shp-contents-dtl="[{&quot;key&quot;:&quot;img_url&quot;,&quot;value&quot;:&quot;https://phinf.pstatic.net/checkout.phinf/20250622_223/1750575630003uHPSd_JPEG/image.jpg&quot;}]"><div class="_9oeeh3ukt7"><div class="_3P12lHpgMB"><a href="#" class="_3qc99-a3Vq" aria-expanded="false"><span cl